# ``Box``

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import pySecDec as psd # Import pySecDec

In this notebook, we experiment with computing a 1-loop box integral:

$$
I_4^{D=4-2\epsilon}(0,0,0,0;s_{12},s_{23},m^2,m^2,m^2,m^2)
$$

### Generating an integral library

First, we will compute the first, ``U``, and second, ``F``, Symmanzik polynomials

In [ ]:
li = psd.LoopIntegralFromGraph(
    internal_lines = [['m',[1,2]],['m',[2,3]],['m',[3,4]],['m',[4,1]]],
    external_lines = [['p1',1],['p2',2],['p3',3],['p4',4]],
    replacement_rules = [
        ('p1*p1', 0),
        ('p2*p2', 0),
        ('p3*p3', 0),
        ('p4*p4', 0),
        ('p1*p2', 's12/2'),
        ('p3*p4', 's12/2'),
        ('p2*p3', 's23/2'),
        ('p1*p4', 's23/2'),
        ('p2*p4', '-s12/2-s23/2'),
        ('p1*p3', '-s12/2-s23/2'),
        ('m**2', 'msq')
    ]
)
print('U:',li.exponentiated_U)
print('F:',li.exponentiated_F)

We now generate the C++ code for the box integral, using ``loop_package``:

In [ ]:
! rm -rf box
Mandelstam_symbols = ['s12','s23']
mass_symbols = ['msq']
psd.loop_package(
    name = 'box',
    loop_integral = li,
    real_parameters = Mandelstam_symbols + mass_symbols,
    requested_orders = [0],
    decomposition_method = 'geometric'
)

Next, we build the integral library, ready for numerical integration.

In [ ]:
! make -C box > /dev/null 2>&1

### Calling the integral library and manipulating the result

We now have an integral library ``box/box_pylink.so`` which we can load into python and call to evaluate our integral.

In [ ]:
box = psd.integral_interface.IntegralLibrary('box/box_pylink.so')

The library will return: 
* the integral result without prefactors, 
* the prefactor
* the integral result with prefactors. 

Usually, we only care about the result with the prefactor, so we will use only that part of the output. In this case the prefactor is also just ``1``.

We begin by evaluating our integral at the point ``s=4.1, t=-2.2, msq=1``.

In [ ]:
_, _, result = box([4.1,-2.2,1.])
print(result)

The result returned by our library is just a string encoding the full result, it depends on all regulators and also tells us the numerical uncertainy on the result. We have convenience functions for converting the output to a format compatible with various symbolic libraries. Here we will use the sympy output format.

In [ ]:
result_sympy = psd.integral_interface.series_to_sympy(result)

In [ ]:
print('res:', result_sympy[0]) # Result
print('err:', result_sympy[1]) # Numerical uncertainty 

We can pick out the finite part of the result using sympy's ``Poly`` module:
* First we strip the trailing ``+ O(eps)`` (Big-O notation) from the string.
* Next, we convert the result to a sympy expression and separate the real and imaginary part

In [ ]:
if result_sympy[0].endswith('+ O(eps)'):
    result_sympy_stripped = sp.sympify(result_sympy[0][:-8]).as_real_imag()
print(result_sympy_stripped[0])

### Plotting the result for different values of the real_parameters

We can also numerically compute our integral at several points, by repeatedly calling the library.

In [ ]:
xx = []
yy_re = []
yy_im = []
for ss in np.arange(0.01,10,0.1):
    _, _, result = box([ss,-2.2,1])
    xx.append(ss)
    result_sympy = psd.integral_interface.series_to_sympy(result)
    result_sympy_stripped = sp.sympify(result_sympy[0][:-8]).as_real_imag()
    yy_re.append(result_sympy_stripped[0])
    yy_im.append(result_sympy_stripped[1])

In [ ]:
plt.plot(xx,yy_re,marker='o')
plt.plot(xx,yy_im,marker='x')